https://habr.com/ru/articles/830396/

https://huggingface.co/facebook/opt-350m

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
from datasets import load_dataset

In [3]:
from trl import SFTTrainer, SFTConfig

In [4]:
import torch
import torch.backends.cuda
import torch.nn.attention

## load

In [5]:
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m", clean_up_tokenization_spaces=True)

In [6]:
model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")

## sample

In [47]:
x = "Hello friend, how much is a"

In [48]:
inputs = tokenizer(x, return_tensors="pt")

In [49]:
inputs

{'input_ids': tensor([[    2, 31414,  1441,     6,   141,   203,    16,    10]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [50]:
model_outputs = model.generate(**inputs, max_new_tokens=16, return_dict_in_generate=True, output_scores=True)

In [51]:
model_outputs

GenerateDecoderOnlyOutput(sequences=tensor([[    2, 31414,  1441,     6,   141,   203,    16,    10,   205,   425,
            13,    10,  1763,     9,   209,   116, 50118,   100,   437,    45,
           686,     6,    38,   437]]), scores=(tensor([[-19.8979, -18.4723, -12.5715,  ..., -18.8881, -20.1659, -20.4875]]), tensor([[-17.0899, -15.2750,  -8.2056,  ..., -15.0639, -16.8087, -16.7109]]), tensor([[-5.4859, -6.4142,  4.9815,  ..., -5.8473, -8.7831, -8.3234]]), tensor([[-22.3065, -22.9499, -15.3558,  ..., -22.2332, -24.4896, -23.6655]]), tensor([[-22.6722, -21.3150, -16.0663,  ..., -22.0091, -23.8076, -21.8936]]), tensor([[-12.8802, -11.0886,   1.7622,  ..., -12.4646, -13.8138, -12.9095]]), tensor([[-24.8371, -24.0937, -17.4236,  ..., -23.3802, -26.6276, -24.4962]]), tensor([[-14.8093, -13.3966,  -1.3259,  ..., -13.0774, -15.2178, -13.7256]]), tensor([[ -9.4803, -10.9797,   5.3210,  ..., -12.6560, -12.4107, -12.7988]]), tensor([[ 9.3513,  9.0060, 17.4618,  ...,  8.6464,  8.4293,  6

In [52]:
generated_tokens_ids = model_outputs.sequences[0]

In [53]:
generated_tokens_ids

tensor([    2, 31414,  1441,     6,   141,   203,    16,    10,   205,   425,
           13,    10,  1763,     9,   209,   116, 50118,   100,   437,    45,
          686,     6,    38,   437])

In [54]:
tokenizer.decode(generated_tokens_ids)

"</s>Hello friend, how much is a good price for a pair of these?\nI'm not sure, I'm"

## train

In [55]:
dataset = load_dataset("imdb", split="train")

In [56]:
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

https://huggingface.co/docs/trl/en/sft_trainer

In [57]:
sft_config = SFTConfig(
    dataset_text_field="text",
    max_seq_length=512,
    output_dir="/tmp",
)

In [58]:
trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    args=sft_config,
)    

/home/nick/.conda/envs/transformers/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [59]:
with torch.backends.cuda.sdp_kernel(enable_flash=True, enable_math=False, enable_mem_efficient=False):
    trainer.train()

/home/nick/.conda/envs/transformers/lib/python3.10/contextlib.py:103: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)


OutOfMemoryError: CUDA out of memory. Tried to allocate 128.00 MiB. GPU 0 has a total capacity of 11.73 GiB of which 29.06 MiB is free. Including non-PyTorch memory, this process has 11.59 GiB memory in use. Of the allocated memory 11.40 GiB is allocated by PyTorch, and 26.95 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)